# Comparing ERA5 and 449MHz profiler data (from Zarr)
Making a comparison of ERA5 and 449MHz profiler data from Zarr stores (pilot study).

### Imports

In [3]:
import xarray as xr

lustre_scratch  = "/lustre/desc1/scratch/myasears"

### Read in zarr stores

In [4]:
era5_test_zarr = xr.open_zarr(f"{lustre_scratch}/2023_M2HATS/era5_M2HATS_ISS2.zarr")
prof449Mhz_test_zarr = xr.open_zarr(f"{lustre_scratch}/2023_M2HATS/prof449_M2HATS_ISS1_winds30.zarr")

### Interpolate datasets to the same grid
We want an hourly time grid + a 200m height grid. ERA5 is on time, but not height; prof449 is on height, but not time. 

### 449 interpolation

In [10]:
# Interpolate the ERA5 dataset onto the 449 time grid
prof449_interp = prof449Mhz_test_zarr.interp(time=era5_test_zarr['time'])

### ERA5 interpolation

Oof... this has been rough. I'm going to return to this next week and interpolate the values onto a height grid before saving it to Zarr, because it's quite a headache to do so with the entire dataset.

In [26]:
era5_test_zarr['height_msl']

<xarray.DataArray 'height_msl' (time: 1896, pressure: 37)> Size: 281kB
dask.array<open_dataset-height_msl, shape=(1896, 37), dtype=float32, chunksize=(1, 37), chunktype=numpy.ndarray>
Coordinates:
  * pressure   (pressure) float64 296B 1.0 2.0 3.0 5.0 ... 950.0 975.0 1e+03
    latitude   float64 8B ...
    longitude  float64 8B ...
  * time       (time) datetime64[ns] 15kB 2023-07-11 ... 2023-09-27T23:00:00
Attributes:
    long_name:  Height above mean sea level
    units:      meters

In [29]:
# Do some finagling to get ERA5 onto an interpolated height grid 
# ('height' is a time-dependent data variable -- ERA5 is run on constant pressure levels.)
target_heights = prof449_interp['height_msl']

era5_test_zarr = era5_test_zarr.assign_coords(
    height_msl=(("time", "pressure"), era5_test_zarr['height_msl'].data)
)

<xarray.Dataset> Size: 1MB
Dimensions:       (time: 1896, pressure: 37)
Coordinates:
    height_msl    (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
  * pressure      (pressure) float64 296B 1.0 2.0 3.0 5.0 ... 950.0 975.0 1e+03
    latitude      float64 8B ...
    longitude     float64 8B ...
  * time          (time) datetime64[ns] 15kB 2023-07-11 ... 2023-09-27T23:00:00
Data variables:
    geopotential  (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    w_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    v_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    u_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.8.1
    CONVERSION_PLATFORM:  Linux r2i4n28 4.12.14-95.51-default #1 SMP Fri Apr ...
    CONVERSION_DATE:      Fri Oct  6 09:19:55 MDT 2023
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Fri Oct  6 09:20:11 2023: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 5.0.3 (Homepage = http://n...

In [31]:
ds_swap = era5_test_zarr.swap_dims({"pressure": "height_msl"})

ValueError: replacement dimension 'height_msl' is not a 1D variable along the old dimension 'pressure'

In [30]:
ds_time_on_fixed_height = era5_test_zarr.interp(
    coords={"height_msl": target_heights},
    kwargs={"fill_value": "extrapolate"}
)

ValueError: Dimensions {'height_msl'} do not exist. Expected one or more of FrozenMappingWarningOnValuesAccess({'time': 1896, 'pressure': 37})

In [18]:
era5_test_zarr

<xarray.Dataset> Size: 1MB
Dimensions:       (time: 1896, pressure: 37)
Coordinates:
  * pressure      (pressure) float64 296B 1.0 2.0 3.0 5.0 ... 950.0 975.0 1e+03
    latitude      float64 8B ...
    longitude     float64 8B ...
  * time          (time) datetime64[ns] 15kB 2023-07-11 ... 2023-09-27T23:00:00
Data variables:
    height_msl    (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    geopotential  (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    w_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    v_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    u_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.8.1
    CONVERSION_PLATFORM:  Linux r2i4n28 4.12.14-95.51-default #1 SMP Fri Apr ...
    CONVERSION_DATE:      Fri Oct  6 09:19:55 MDT 2023
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Fri Oct  6 09:20:11 2023: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 5.0.3 (Homepage = http://n...